## Compute ipsilateral thalamo-cortical functional connectivity


Ipsilateral functional connectivity (FC) was computed between left (or right) thalamus and left (or right) cortex. Paths are specific to the user.

The code was written by Oualid Benkarim and modified by Valeria Kebets.

In [2]:
import os
import argparse
import numpy as np
import pandas as pd
import nibabel as nib

from brainspace.mesh.mesh_io import read_surface
from brainspace.mesh import mesh_elements as me
from brainspace.utils.parcellation import reduce_by_labels

In [3]:
# Specify files & directories

pth_data = '/host/oncilla/local_raid'
pth_out = os.path.join(pth_data,'/valeria/sensory_gradients')
pth_tools = 'https://raw.githubusercontent.com/valkebets/thalamocortical_gradients/master/data/'

pth_mask = os.path.join(pth_out, 'thalamus_intersectLR_mask_1.nii') # Using symmetric thalamus mask
pth_surf = os.path.join(pth_tools, 'my_conte_10k.vtp') # surface map (20484 vertices)
pth_demog = os.path.join(pth_tools, 'abide_func_fn_abideI.csv') # tabular data for ABIDE-I participants
out_fname = 'ipsilateral_FC_symmetric-thal_Schaefer400_abide1_N210.npz' # output filename

In [4]:
def fill_empty_ts(ts, dist_adj):
    
    """Fill zero timeseries (if any) with average timeseries of immediate neighbors."""

    mk = np.all(ts == 0, axis=1)
    while mk.sum() > 0:
        idx_miss = np.where(mk)[0]
        x2 = [[idx, mk[dist_adj[idx].tocoo().col].sum(),
               dist_adj[idx].tocoo().col] for idx in idx_miss]

        idx = np.argmin(idx_miss)
        tidx, tn, tneigh = x2[idx]

        tneigh = tneigh[~mk[tneigh]]
        ts[tidx] = ts[tneigh].mean(0)
        mk[tidx] = False
    return ts

In [5]:
def get_con(sid, sid_orig, site, parc, mask, dist_ring=None, smooth=3):
    
    """Compute ipsilateral FC between thalamus and cortex."""

    print(f'Subject {sid}')

    pth_ts = os.path.join(pth_data, f'abide1/01_fs_processed/{sid}/hcp_processed')

    # Load thalamus mask
    try:
        thal_mask = np.asanyarray(nib.load(pth_mask).dataobj)
    except:
        return None, None, None

    # Load subcortical timeseries
    pth_sub = os.path.join(pth_ts, f'{site}_{sid_orig}_func_'
                                f'AtlasSubcortical_s{smooth}.nii.gz')
    try:
        ts_sub = np.asanyarray(nib.load(pth_sub).dataobj)
    except:
        return None, None, None

    # Divide thalamus timeseries into left/right
    ts_thal_l = ts_sub[np.isin(thal_mask, [10])]
    ts_thal_r = ts_sub[np.isin(thal_mask, [49])]

    # Get number of voxels in left/right thalamus
    nl, nr = np.sum(thal_mask == 10), np.sum(thal_mask == 49)

    # Load cortical timeseries
    pth_ctx = os.path.join(pth_ts, f'{site}_{sid_orig}_func.dtseries.nii')
    try:
        ts_ctx = np.asanyarray(nib.load(pth_ctx).dataobj).T
    except:
        return None, None, None

    # Remove midline
    ts_ctx = ts_ctx[:mask.size]
    ts_ctx = ts_ctx[mask]

    # Fill remaining timeseries if zero (happens with small number of subjects)
    if dist_ring is not None:
        ts_ctx = fill_empty_ts(ts_ctx, dist_ring)
       
    # Average timeseries per cortical parcel
    ts_ctx = reduce_by_labels(ts_ctx, parc[mask], red_op='mean', axis=1, dtype=np.float32)

    # Divide cortex timeseries into left/right
    ts_ctx_l = ts_ctx[:200,:] 
    ts_ctx_r = ts_ctx[200:,:] 
    
    # Compute FC between thalamus/cortex within left/right hemispheres
    fc_l = np.corrcoef(ts_ctx_l, y=ts_thal_l).astype(np.float32)
    fc_r = np.corrcoef(ts_ctx_r, y=ts_thal_r).astype(np.float32)
    
    # Fisher Z transform
    fcz_l = np.arctanh(fc_l)
    fcz_l[~np.isfinite(fcz_l)] = 0    
    fcz_r = np.arctanh(fc_r)
    fcz_r[~np.isfinite(fcz_r)] = 0

    
    return fc_l, fc_r, fcz_l, fcz_r, nl, nr

In [6]:
def build_con(df_phen, parc, smooth=6, dist_ring=None):
    
    """Stack FC matrices of all participants."""

    # Initialize matrices with stacked FC + num voxels left/right thalamus
    fcs_l = []
    fcs_r = []
    nl = nr = 0
    mask_sel = []
    
    for sid, rw in df_phen.iterrows():
        site = rw.Site
        sid_orig = f'00{rw.OrgID}'

        fc_l, fc_r, fcz_l, fcz_r, nl, nr = get_con(sid, sid_orig, site, parc, mask, dist_ring=dist_ring, smooth=smooth)

        if fc_l is None or fcz_ is None or nl is None or nr is None:
            print(f'Discarding: {site} {sid}')
            mask_sel.append(False)
            continue

        mask_sel.append(True)
        fcs_l.append(fc_l)
        fcs_r.append(fc_r)
        fczs_l.append(fcz_l)
        fczs_r.append(fcz_r)

    
    return np.stack(fcs_l,0), np.stack(fcs_r,0), np.stack(fczs_l,0), np.stack(fczs_r,0), np.array(mask_sel), nl, nr

In [5]:
# Choose parcellation, load surface

parcel_choices = [100, 200, 300, 400, 500, 600, 800]

parser = argparse.ArgumentParser()
parser.add_argument('-p', '--parcel', help='number of parcels', type=int,
                    choices=parcel_choices, default=400)
parser.add_argument('-s', '--smooth', help='smooth radius', type=int,
                    choices=[3, 6], default=6)
parser.add_argument('-o', '--opth', help='filename used to save matrices',
                    type=str, default=None)

#args = parser.parse_args() # error in jupyter but not on python
args = parser.parse_args(args=[])

n_parcels = args.parcel
smooth_radius = args.smooth
opth_matrix = args.opth

# Load surface
surf = read_surface(pth_surf)
mask = surf.PointData['mask'] == 1
dist_ring = me.get_immediate_distance(surf, mask=mask)

# Filename to store connectivity matrices
if opth_matrix is None:
    opth_matrix = os.path.join(pth_out, out_fname)

# Load Schaefer parcellation
parc = surf.PointData[f'schaefer{n_parcels}_yeo7']
n_ctx = np.unique(parc[mask]).size
print(f'Using Schaefer parcellation with {n_parcels} parcels')

In [6]:
# Load demographic data

df =  pd.read_csv(pth_demog, header=0, index_col=[0])

# Select columns of interest
df_phen = df[['OrgID','Group','Site','Func_MeanFD']]

# Remove subjects with high head motion (FD > 0.3)
df_phen = df_phen.loc[(df_phen['Func_MeanFD'] < 0.3)]

# Remove problematic subject manually
df_phen = df_phen[df_phen.index != 32183]

df_phen

OrgID    Group  Site  Func_MeanFD
ID                                      
32319  50002      ASD  Pitt     0.116828
32321  50004      ASD  Pitt     0.127745
32581  50006      ASD  Pitt     0.070143
32323  50007      ASD  Pitt     0.151246
32324  50008      ASD  Pitt     0.169275
...      ...      ...   ...          ...
32252  51152  CONTROL   NYU     0.045205
32253  51153  CONTROL   NYU     0.045803
32254  51154  CONTROL   NYU     0.052091
32255  51155  CONTROL   NYU     0.043319
32256  51156  CONTROL   NYU     0.100715

[210 rows x 4 columns]

In [8]:
# Build FC matrices for ASD and TD participants

fcs_l, fcs_r, fczs_l, fczs_r, mask_sel, nl, nr = build_con(df_phen, parc, smooth=smooth_radius, dist_ring=dist_ring)

print(f'N points: Cortex = {n_ctx}, Thalamus = [{nl} left, '
      f'{nr} right]')

Subject 32319
Subject 32321
Subject 32581
Subject 32323
Subject 32324
Subject 32325
Subject 32326
Subject 32327
Subject 32328
Subject 32330
Subject 32331
Subject 32332
Subject 32582
Subject 32335
Subject 32337
Subject 32338
Subject 32340
Subject 32341
Subject 32367
Subject 32343
Subject 32345
Subject 32346
Subject 32347
Subject 32348
Subject 32350
Subject 32352
Subject 32353
Subject 32354
Subject 32355
Subject 32356
Subject 32357
Subject 32358
Subject 32359
Subject 32360
Subject 32362
Subject 32363
Subject 32364
Subject 32370
Subject 32372
Subject 32524
Subject 32525
Subject 32526
Subject 32527
Subject 32528
Subject 32529
Subject 32530
Subject 32531
Subject 32532
Subject 32533
Subject 32536
Subject 32537
Subject 32538
Subject 32539
Subject 32540
Subject 32541
Subject 32542
Subject 32543
Subject 32545
Subject 32546
Subject 32547
Subject 32548
Subject 32549
Subject 32550
Subject 32551
Subject 32552
Subject 32553
Subject 32554
Subject 32556
Subject 32557
Subject 32558
Subject 32559
Subjec

In [10]:
# Save FC data

print()
print(f'Saving matrices to: {opth_matrix}')
np.savez_compressed(opth_matrix, fcs_l=fcs_l, fcs_r=fcs_r, fczs_l=fcs_l, fczs_r=fcs_r, 
                    mask=mask_sel, n_ctx=n_ctx, nl=n_thal_l, nr=n_thal_r)


Saving matrices to: /host/oncilla/local_raid/valeria/sensory_gradients/individual_FC_Schaefer400_perhemi_symmetric-thal_abide1_test.npz
